In [12]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: not using Google CoLab


In [16]:
#Importing initial libraries
import numpy as np
import random
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
if COLAB:
    df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/df.csv', low_memory=False)
else:
    df = pd.read_csv("Data/final_dataset/df.csv", low_memory=False)

df.rename(columns={'Unnamed: 0':'patient_num'},inplace= True)
df.set_index('patient_num',inplace= True)
#Add the Binary Severity Metric
df['Severity'] = (df['LOS']>= 7).astype(int)

#Make sure all column names are valid
import re
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

#Make sure we dont have any duplicate column names
while(len(set(df.columns)) != len(df.columns)):
  cols = pd.Series(df.columns)
  for dup in df.columns[df.columns.duplicated(keep=False)]: 
      cols[df.columns.get_loc(dup)] = ([dup + 'A' + str(d_idx) 
                                       if d_idx != 0 
                                       else dup 
                                       for d_idx in range(df.columns.get_loc(dup).sum())]
                                      )
  df.columns = cols
  try:
    df.set_index('patient_num',inplace= True)
  except:
    print('patient_num already index')

#Remove 0 variance features because they just take up space
from sklearn.feature_selection import VarianceThreshold
selector_var_thresh = VarianceThreshold(threshold = 0)
selector_var_thresh.fit(df)
df = df[df.columns[selector_var_thresh.get_support(indices=True)]]

#Categorical variable identification
cat = []
for col in df.columns:
    if np.array_equal(df[col].unique(),[0., 1.]) or np.array_equal(df[col].unique(),[1., 0.]):
        cat.append(col)
df_cat = df[cat]
df_num = df.drop(cat, axis = 1)

#################################################################################
#Choose the scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_num = pd.DataFrame(data = scaler.fit_transform(df_num), index = df_num.index, columns = df_num.columns)
df = pd.merge(df_num,df_cat, left_index= True,right_index= True, how = 'inner')

#Stratified train test split
from sklearn.model_selection import StratifiedShuffleSplit
#Create bins
import math
strat = list(df.LOS)
strat.sort()
bin = []
for i in range(1,5):
    bin.append(strat[math.ceil(i/5*len(strat))])

df['LOS_cat'] = pd.cut(
    df['LOS'],
    bins = [-np.inf,bin[0],bin[1],bin[2],bin[3],np.inf],
    labels = [1,2,3,4,5])

from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df, df['LOS_cat']):
    train_set_full = df.iloc[train_index]
    test_set = df.iloc[test_index]

val_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in val_split.split(train_set_full, train_set_full['LOS_cat']):
    train_set = df.iloc[train_index]
    val_set = df.iloc[test_index]

train_set_full.drop('LOS_cat',axis = 1,inplace = True)
test_set.drop('LOS_cat',axis = 1,inplace = True)
train_set.drop('LOS_cat',axis = 1,inplace = True)
val_set.drop('LOS_cat',axis = 1,inplace = True)

#Get validation, training and test sets
x_train = train_set.drop(['LOS','Severity'],axis = 1)
y_train = train_set['Severity']
x_val = val_set.drop(['LOS','Severity'],axis = 1)
y_val = val_set['Severity']
x_test = test_set.drop(['LOS','Severity'],axis = 1)
y_test = test_set['Severity']

if COLAB:
    x_train.to_csv('/content/drive/MyDrive/Colab Notebooks/Data/x_train.csv' )
    y_train.to_csv('/content/drive/MyDrive/Colab Notebooks/Data/y_train.csv' )
    x_val.to_csv('/content/drive/MyDrive/Colab Notebooks/Data/x_val.csv' )
    y_val.to_csv('/content/drive/MyDrive/Colab Notebooks/Data/y_val.csv' )
    x_test.to_csv('/content/drive/MyDrive/Colab Notebooks/Data/x_test.csv' )
    y_test.to_csv('/content/drive/MyDrive/Colab Notebooks/Data/y_test.csv' )
else:
    x_train.to_csv('Data/final_dataset/x_train.csv')
    y_train.to_csv('Data/final_dataset/y_train.csv')
    x_val.to_csv('Data/final_dataset/x_val.csv')
    y_val.to_csv('Data/final_dataset/y_val.csv')
    x_test.to_csv('Data/final_dataset/x_test.csv')
    y_test.to_csv('Data/final_dataset/y_test.csv')

patient_num already index
patient_num already index
patient_num already index


C:\Users\zfarahany\AppData\Local\Temp\ipykernel_14784\1095871475.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set_full.drop('LOS_cat',axis = 1,inplace = True)
C:\Users\zfarahany\AppData\Local\Temp\ipykernel_14784\1095871475.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set.drop('LOS_cat',axis = 1,inplace = True)
C:\Users\zfarahany\AppData\Local\Temp\ipykernel_14784\1095871475.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi